<a href="https://colab.research.google.com/github/MasaAsami/LTsandox/blob/main/notebooks/DEA_simpleAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install streamlit

In [4]:
# --- app を書き出し ---
app_code = r"""
import streamlit as st
import numpy as np
import pandas as pd
from scipy.optimize import linprog

st.set_page_config(page_title="DEA Simple App (Streamlit)", layout="wide")

# ---------- solver フォールバック ----------
def _linprog_safe(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, bounds=None):
    for m in ("highs", "highs-ipm", "highs-ds"):
        res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
                      bounds=bounds, method=m)
        if res.success:
            return res
    return res

# ---------- DEA ソルバ ----------
def dea_solve(df, inputs, outputs, orientation="input", rts="VRS", dmu_col=None):
    assert len(inputs) > 0 and len(outputs) > 0, "入力列・出力列を最低1つずつ選んでください。"
    work = df.copy()
    cols = inputs + outputs + ([dmu_col] if dmu_col else [])
    work = work[cols]

    # 数値化 & 検査
    for c in inputs + outputs:
        work[c] = pd.to_numeric(work[c], errors="coerce")
    if work[inputs + outputs].isnull().any().any():
        raise ValueError("非数値（NaN）が含まれています。")
    if (work[inputs + outputs] < 0).any().any():
        raise ValueError("DEAでは負値は扱えません（0以上にしてください）。")

    # DMU 名
    names = work[dmu_col].astype(str).tolist() if dmu_col and dmu_col in work.columns \
            else [f"DMU_{i+1}" for i in range(len(work))]

    X = work[inputs].to_numpy().T  # m x N
    Y = work[outputs].to_numpy().T # s x N
    m, N = X.shape[0], X.shape[1]
    s = Y.shape[0]

    results = []
    lambda_mat = np.full((N, N), np.nan)

    for o in range(N):
        if orientation == "input":
            # min θ, 変数 = [θ, λ1..λN]
            c = np.concatenate(([1.0], np.zeros(N)))
            A_ub, b_ub = [], []
            # 入力: sum_j λ_j x_ij - θ x_io <= 0
            for i in range(m):
                A_ub.append(np.concatenate(([-X[i, o]], X[i, :])))
                b_ub.append(0.0)
            # 出力: -sum_j λ_j y_rj <= -y_ro
            for r in range(s):
                A_ub.append(np.concatenate(([0.0], -Y[r, :])))
                b_ub.append(-Y[r, o])
            A_ub, b_ub = np.array(A_ub), np.array(b_ub)

            # VRS(BCC): ∑λ=1,  CRS(CCR): 制約なし
            if rts.upper() == "VRS":
                A_eq = np.concatenate(([0.0], np.ones(N)))[None, :]
                b_eq = np.array([1.0])
            else:
                A_eq = None; b_eq = None

            bounds = [(0, None)] + [(0, None)] * N
            res = _linprog_safe(c, A_ub, b_ub, A_eq, b_eq, bounds)
            if not res.success:
                results.append({"DMU": names[o], "orientation": "input", "rts": rts,
                                "status": f"failed: {res.message}"})
                continue

            theta   = float(res.x[0])
            lambdas = res.x[1:]
            lambda_mat[o, :] = lambdas

            x_hat = X @ lambdas
            y_hat = Y @ lambdas
            s_minus = np.where(np.abs(theta * X[:, o] - x_hat) < 1e-10, 0.0, theta * X[:, o] - x_hat)
            s_plus  = np.where(np.abs(y_hat - Y[:, o])         < 1e-10, 0.0, y_hat - Y[:, o])

            peers = [(names[j], float(lambdas[j])) for j in range(N) if lambdas[j] > 1e-8]
            row = {"DMU": names[o], "orientation": "input", "rts": rts, "status": "optimal",
                   "theta": theta, "efficiency": theta, "num_peers": len(peers),
                   "peers(top10)": "; ".join([f"{n}:{w:.4f}" for n, w in peers[:10]])}
            for idx, col in enumerate(inputs):
                row[f"target_in_{col}"] = float(x_hat[idx]); row[f"slack_in_{col}"] = float(max(s_minus[idx], 0.0))
            for idx, col in enumerate(outputs):
                row[f"target_out_{col}"] = float(y_hat[idx]); row[f"slack_out_{col}"] = float(max(s_plus[idx], 0.0))
            results.append(row)

        else:
            # max φ → min -φ, 変数 = [φ, λ1..λN]
            c = np.concatenate(([-1.0], np.zeros(N)))
            A_ub, b_ub = [], []
            # 入力: sum_j λ_j x_ij <= x_io
            for i in range(m):
                A_ub.append(np.concatenate(([0.0], X[i, :])))
                b_ub.append(X[i, o])
            # 出力: -sum_j λ_j y_rj + φ y_ro <= 0
            for r in range(s):
                A_ub.append(np.concatenate(([Y[r, o]], -Y[r, :])))
                b_ub.append(0.0)
            A_ub, b_ub = np.array(A_ub), np.array(b_ub)

            if rts.upper() == "VRS":
                A_eq = np.concatenate(([0.0], np.ones(N)))[None, :]
                b_eq = np.array([1.0])
            else:
                A_eq = None; b_eq = None

            bounds = [(0, None)] + [(0, None)] * N
            res = _linprog_safe(c, A_ub, b_ub, A_eq, b_eq, bounds)
            if not res.success:
                results.append({"DMU": names[o], "orientation": "output", "rts": rts,
                                "status": f"failed: {res.message}"})
                continue

            phi    = float(res.x[0])
            lambdas = res.x[1:]
            lambda_mat[o, :] = lambdas

            x_hat = X @ lambdas
            y_hat = Y @ lambdas
            s_minus = np.where(np.abs(X[:, o] - x_hat)       < 1e-10, 0.0, X[:, o] - x_hat)
            s_plus  = np.where(np.abs(phi * Y[:, o] - y_hat) < 1e-10, 0.0, phi * Y[:, o] - y_hat)

            eff = (1.0 / phi) if phi > 0 else np.nan
            peers = [(names[j], float(lambdas[j])) for j in range(N) if lambdas[j] > 1e-8]
            row = {"DMU": names[o], "orientation": "output", "rts": rts, "status": "optimal",
                   "phi": phi, "efficiency": eff, "num_peers": len(peers),
                   "peers(top10)": "; ".join([f"{n}:{w:.4f}" for n, w in peers[:10]])}
            for idx, col in enumerate(inputs):
                row[f"target_in_{col}"] = float(x_hat[idx]); row[f"slack_in_{col}"] = float(max(s_minus[idx], 0.0))
            for idx, col in enumerate(outputs):
                row[f"target_out_{col}"] = float(y_hat[idx]); row[f"slack_out_{col}"] = float(max(s_plus[idx], 0.0))
            results.append(row)

    res_df = pd.DataFrame(results)
    lam_df = pd.DataFrame(lambda_mat, columns=names, index=names)
    return res_df, lam_df

# ---------- サンプル ----------
def sample_df():
    return pd.DataFrame({
        "DMU": list("ABCDEF"),
        "labor":  [5, 7, 4, 9, 6, 8],
        "capital":[2, 3, 2, 5, 4, 2],
        "output": [10,12, 9,14,11,15],
    })

# ---------- UI ----------
if "df" not in st.session_state:
    st.session_state.df = pd.DataFrame()

st.title("DEA Simple App (CCR/BCC) — Streamlit")
st.markdown("- 変数は **0以上**（負値不可）\n- 入力志向: θ 最小化 / 出力志向: φ 最大化（表示は 1/φ）\n- **CRS = CCR**, **VRS = BCC**")

colL, colR = st.columns(2)

with colL:
    up = st.file_uploader("CSVファイル（UTF-8）", type=["csv"])
    if st.button("サンプルデータを読み込む"):
        st.session_state.df = sample_df()
    elif up is not None:
        st.session_state.df = pd.read_csv(up)

    df = st.session_state.df
    if not df.empty:
        st.subheader("プレビュー")
        st.dataframe(df.head(100), use_container_width=True)

with colR:
    df = st.session_state.df
    if not df.empty:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        all_cols = df.columns.tolist()

        model = st.radio("モデル（Returns to Scale）", ["BCC (VRS)", "CCR (CRS)"], index=0, horizontal=True)
        rts = "VRS" if model.startswith("BCC") else "CRS"

        dmu_col = st.selectbox("DMU列（任意）", ["（なし）"] + all_cols,
                               index=(all_cols.index("DMU")+1 if "DMU" in all_cols else 0))
        dmu_col = None if dmu_col == "（なし）" else dmu_col

        init_inputs = numeric_cols[:-1] if len(numeric_cols) >= 2 else numeric_cols
        init_outputs = numeric_cols[-1:] if len(numeric_cols) >= 1 else []

        inputs_sel  = st.multiselect("入力列（複数可）", numeric_cols, default=init_inputs)
        outputs_sel = st.multiselect("出力列（複数可）", numeric_cols, default=init_outputs)

        orientation = st.radio("志向", ["input", "output"], index=0, horizontal=True)

        if st.button("DEAを実行", type="primary"):
            try:
                res_df, lam_df = dea_solve(df, inputs_sel, outputs_sel, orientation, rts, dmu_col)
                st.success("最適化に成功しました。")
                st.subheader("結果（効率性・ターゲット・スラック・ピア）")
                st.dataframe(res_df, use_container_width=True)
                st.subheader("λ（ピア重み）行=評価DMU / 列=参照DMU")
                st.dataframe(lam_df, use_container_width=True)
                st.download_button("結果CSVをダウンロード",
                                   res_df.to_csv(index=False).encode("utf-8-sig"),
                                   "dea_results.csv", "text/csv")
                st.download_button("λ（ピア重み）CSVをダウンロード",
                                   lam_df.to_csv().encode("utf-8-sig"),
                                   "dea_lambdas.csv", "text/csv")
            except Exception as e:
                st.error(f"エラー: {e}")
    else:
        st.info("左側で CSV をアップロードするか、サンプルデータを読み込んでください。")
"""
with open("dea_app.py", "w", encoding="utf-8") as f:
    f.write(app_code)


In [3]:
# --- Colab のプロキシURLを先に取得（baseUrlPath を設定するため） ---
from google.colab import output
from urllib.parse import urlparse
public_url = output.eval_js("google.colab.kernel.proxyPort(8501)")
base_path = urlparse(public_url).path  # 例: '/', '/proxy/8501/'

# --- Streamlit を起動（CORS/XSRF 無効 + baseUrlPath を必要に応じて付与） ---
import subprocess, shlex, time, sys

cmd = [
  "streamlit","run","dea_app.py",
  "--server.port","8501",
  "--server.headless","true",
  "--server.address","0.0.0.0",
  "--server.enableCORS","false",
  "--server.enableXsrfProtection","false",
  "--server.fileWatcherType","none",
  "--browser.gatherUsageStats","false",
]

if base_path and base_path != "/":
    cmd += ["--server.baseUrlPath", base_path.strip("/")]

print("Launching:", " ".join(shlex.quote(c) for c in cmd))
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# 少し待ってURL表示
time.sleep(3)
print("App URL:", public_url)
print("↑ をクリックしてアプリを開いてください。")

# ログを随時表示（デバッグ用）
import threading
def _tail(p):
    for line in iter(p.stdout.readline, ""):
        sys.stdout.write(line)
threading.Thread(target=_tail, args=(proc,), daemon=True).start()

Launching: streamlit run dea_app.py --server.port 8501 --server.headless true --server.address 0.0.0.0 --server.enableCORS false --server.enableXsrfProtection false --server.fileWatcherType none --browser.gatherUsageStats false
App URL: https://8501-m-s-2ltjnvcmaw7u1-a.asia-east1-1.prod.colab.dev
↑ をクリックしてアプリを開いてください。



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

